# Experimentos

Experimentos são a principal forma de agrupar 3 entidades:

1. **Datasets**: conjunto de textos acompanhados de um *target*, representando um problema de classificação ou regressão;
2. **Pipelines**: são capazes de receber um ou mais textos como entrada e produzir uma saída para o problema de classificação/regressão;
3. **Métricas**: permitem avaliar o desempenho de uma ou mais pipelines para o problema em questão;

A biblioteca `aibox-nlp` disponibiliza diferentes métodos para construção e execução de experimentos em seu pacote `aibox.nlp.experiments`. É possível construir um experimento instanciando cada um dos componentes individualmente ou através do pacote `aibox.nlp.factory`, que possui facilidades para obter as diversas classes da biblioteca.

In [1]:
import json

import pandas as pd

from aibox.nlp.factory.experiment import SimpleExperimentBuilder

In [2]:
# === Construindo um experimento para classificação no Essay-BR ===
# Por simplicidade, vamos instanciar um experimento
#   para comparar algumas abordagens para classificação
#   da competência 1 do dataset Essay-BR.
builder = SimpleExperimentBuilder()

# Inicialmente, vamos definir o dataset
builder.dataset('essayBR',
                extended=False,
                target_competence='C1')

# Também é possível passar uma instância
#   de um Dataset diretamente:
# builder.custom_dataset(ds)

# Vamos definir o tipo do problema
builder.classification()

# Vamos definir a seed randômica
builder.seed(42)

# Depois, vamos definir algumas métricas
#   que devem ser calculadas
builder.add_metric('precision', average='weighted')
builder.add_metric('recall', average='weighted')
builder.add_metric('f1', average='weighted')
builder.add_metric('kappa')
builder.add_metric('neighborKappa')

# Depois, vamos definir qual a métrica
#   que deve ser utilizar para escolher a
#   melhor pipeline
builder.best_criteria('precision', average='weighted')

# Agora, vamos adicionar algumas pipelines baseadas
#   em extração de característica
builder.add_feature_pipeline(
    features=['readabilityBR',
              'regencyBR',
              'syntacticComplexityBR',
              'textualSimplicityBR'],
    estimators=['svm',
                'etreesClf',
                'lgbmClf',
                'xgbClf'],
    names=['svm+features',
           'etrees+features',
           'lgbm+features',
           'xgb+features'])

# Agora, vamos adicionar algumas pipelines baseadas
#   em outras estratégias de vetorização
builder.add_vectorizer_pipeline('tfidfVectorizer',
                                estimators=['etreesClf',
                                            'lgbmClf',
                                            'xgbClf'],
                                names=['etrees+tfidf',
                                       'lgbm+tfidf',
                                       'xgb+tfidf'],
                                estimators_configs=[dict(n_estimators=20),
                                                    dict(n_estimators=20),
                                                    dict(n_estimators=20)])
builder.add_vectorizer_pipeline('bertVectorizer',
                                estimators=['svm',
                                            'etreesClf',
                                            'lgbmClf',
                                            'xgbClf'],
                                names=['svm+bert',
                                       'etrees+bert',
                                       'lgbm+bert',
                                       'xgb+bert'])

# Vamos aproveitar um conjunto de características
#   pré-extraídos se ele existir:
features = None
try:
    features = pd.read_csv('essay_br_features.csv')
except Exception:
    ...

# Uma vez que tenhamos configurado o experimento,
#   podemos obter uma instância:
experiment = builder.build(features_df=features)

In [3]:
# === Executando o experimento ===
result = experiment.run()

[INFO] [aibox.nlp.experiments.simple_experiment] Setting up experiment...
[INFO] [aibox.nlp.experiments.simple_experiment] Obtaining train and test split...
[INFO] [aibox.nlp.experiments.simple_experiment] Train has 3656 samples, Test has 914 samples.
[INFO] [aibox.nlp.experiments.simple_experiment] Run started.
[INFO] [aibox.nlp.experiments.simple_experiment] Started pipeline "xgb+features" (1/11)


Vetorização:   0%|          | 0/3656 [00:00<?, ?it/s]

Vetorização:   0%|          | 0/914 [00:00<?, ?it/s]

[INFO] [aibox.nlp.experiments.simple_experiment] Started pipeline "lgbm+features" (2/11)


Vetorização:   0%|          | 0/3656 [00:00<?, ?it/s]

...

Vetorização:   0%|          | 0/914 [00:00<?, ?it/s]

[INFO] [aibox.nlp.experiments.simple_experiment] Started pipeline "etrees+tfidf" (11/11)


Vetorização:   0%|          | 0/3656 [00:00<?, ?it/s]

Vetorização:   0%|          | 0/914 [00:00<?, ?it/s]

[INFO] [aibox.nlp.experiments.simple_experiment] Run finished in 407.82 seconds.
Best pipeline: svm+bert


In [4]:
# === Inspecionando os resultados ===
# Podemos obter o nome da melhor pipeline:
result.best_pipeline.name

'svm+bert'

In [5]:
# Também podemos listar o valor das melhores métricas:
print(json.dumps({k: v.tolist() for k, v in result.best_metrics.items()},
                 indent=2,
                 ensure_ascii=False))

{
  "Weighted Precision": 0.6892187595367432,
  "Weighted Recall": 0.7013128995895386,
  "Weighted F1-score": 0.6456167101860046,
  "Kappa": 0.3914691209793091,
  "Neighbor Kappa": 0.3914691209793091
}


In [6]:
# Também podemos listar o valor de todas as métricas calculadas:
print(json.dumps({k: {k_: v_.tolist() for k_, v_ in v.items()} 
                  for k, v in result.metrics_history.items()},
                 indent=2,
                 ensure_ascii=False))

{
  "xgb+features": {
    "Weighted Precision": 0.4791104793548584,
    "Weighted Recall": 0.550328254699707,
    "Weighted F1-score": 0.500983476638794,
    "Kappa": 0.09217804670333862,
    "Neighbor Kappa": 0.09217804670333862
  },
  "lgbm+features": {
    "Weighted Precision": 0.49731308221817017,
    "Weighted Recall": 0.5689277648925781,
    "Weighted F1-score": 0.5195298194885254,
    "Kappa": 0.12839876115322113,
    "Neighbor Kappa": 0.12839876115322113
  },
  "etrees+features": {
    "Weighted Precision": 0.48885378241539,
    "Weighted Recall": 0.5831509828567505,
    "Weighted F1-score": 0.49162736535072327,
    "Kappa": 0.08026918023824692,
    "Neighbor Kappa": 0.08026918023824692
  },
  "svm+features": {
    "Weighted Precision": 0.33119142055511475,
    "Weighted Recall": 0.5754923224449158,
    "Weighted F1-score": 0.42042914032936096,
    "Kappa": 0.0,
    "Neighbor Kappa": 0.0
  },
  "xgb+bert": {
    "Weighted Precision": 0.6851080060005188,
    "Weighted Recall": 0

In [7]:
# Por último, também é possível recuperar um DataFrame com as características
#   extraídas para os textos do dataset.
result.extras.df_features.head(3)

,text,adapted_dalechall,adverbs_before_main_verb_ratio,brunet_indice,clauses_per_sentence,cncadc,cncadd,cncall,cncalter,cnccaus,...,wrdnoun,wrdpro,wrdprp1p,wrdprp1s,wrdprp2,wrdprp2p,wrdprp2s,wrdprp3p,wrdprp3s,wrdverb
0,Apropriação cultural significa uma pessoa pode...,4.680675,0.352941,10.521848,2.428571,1.0,4.0,4.0,0.0,6.0,...,39.0,9.0,0.0,0.0,2.0,0.0,2.0,1.0,0.0,20.0
1,A desigualdade social é um problema que afeta ...,4.744804,0.115385,11.063507,2.363636,0.0,17.0,9.0,0.0,9.0,...,62.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
2,Hoje dependemos igualmente da qualidade de ens...,4.831121,0.900000,10.022355,2.500000,0.0,6.0,4.0,1.0,2.0,...,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0


In [8]:
# Podemos salvar os resultados para reutilizar
#   em outro momento.
from pathlib import Path

output = Path('essay_br_features.csv')

if not output.exists():
    result.extras.df_features.to_csv(output,
                                     index=False)